##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=bPJq2qP2KE3u).

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/swift/tutorials/raw_tensorflow_operators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
</table>

# 未加工の TensorFlow 演算子

TensorFlow を基に構築された Swift for TensorFlow の API 設計には、斬新なアプローチが採用されています。API は確立したライブラリから慎重に選択され、新しい言語イディオムと組み合わされています。つまり、すべての TensorFlow API が Swift API として使用できるわけではなく、API のキュレーションを進めるには時間と献身的な取り組みが必要です。とはいえ、お気に入りの TensorFlow 演算子が Swift で使用できなくても、悩むことはありません。TensorFlow Swift ライブラリには、`_Raw` 名前空間の下に、ほとんどの TensorFlow 演算子への透過的なアクセスが提供されています。


まず `TensorFlow` をインポートします。

In [ ]:
import TensorFlow

## 未加工の演算を呼び出す

コード補完機能を使用して、`_Raw` 名前空間から必要な関数を見つけてください。

In [ ]:
_Raw.mul(Tensor([2.0, 3.0]), Tensor([5.0, 6.0]))

## 新しい乗算演算子を定義する

乗算は、すでに演算子 `*` として `Tensor` に提供されていますが、`.*` という新しい名前を付けることにしましょう。Swift では、`extension` 宣言を使用して、メソッドや計算済みのプロパティを遡及的に既存の型に追加できます。

では、extension を宣言して `.*` を `Tensor` に追加し、テンソルの `Scalar` 型が [`Numeric`](https://developer.apple.com/documentation/swift/numeric) に適合する場合に利用できるようにしましょう。

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    static func .* (_ lhs: Tensor, _ rhs: Tensor) -> Tensor {
        return _Raw.mul(lhs, rhs)
    }
}

let x: Tensor<Double> = [[1.0, 2.0], [3.0, 4.0]]
let y: Tensor<Double> = [[8.0, 7.0], [6.0, 5.0]]
x .* y

## ラッピングされた関数の導関数を定義する

未加工の TensorFlow 演算子に Swift API を定義できるだけでなく、Swift のファーストクラスの自動微分を使用できるように微分可能にすることもできます。

`.*` を微分可能にするには、導関数で `@derivative` 属性を使用し、元の関数を `of:` ラベルの属性引数として指定します。`.*` 演算子は、汎用型の `Scalar` が `Numeric` に適合する場合に定義されるため、`Tensor<Scalar>` が `Differentiable` プロトコルに適合されるようにするには不足しています。型安全性を備える Swift では、`@differentiable` 属性に対して、`Scalar` が `TensorFlowFloatingPoint` プロトコルに適合しなければならないという一般制約を課すことが求められるため、`Tensor<Scalar>` は `Differentiable` に適合されるようになります。

```swift
@differentiable(where Scalar: TensorFlowFloatingPoint)
```

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    @differentiable(where Scalar: TensorFlowFloatingPoint)
    static func .* (_ lhs: Tensor,  _ rhs: Tensor) -> Tensor {
        return _Raw.mul(lhs, rhs)
    }
}

extension Tensor where Scalar : TensorFlowFloatingPoint { 
    @derivative(of: .*)
    static func multiplyDerivative(
        _ lhs: Tensor, _ rhs: Tensor
    ) -> (value: Tensor, pullback: (Tensor) -> (Tensor, Tensor)) {
        return (lhs * rhs, { v in
            ((rhs * v).unbroadcasted(to: lhs.shape),
            (lhs * v).unbroadcasted(to: rhs.shape))
        })
    }
}

// Now, we can take the derivative of a function that calls `.*` that we just defined.
gradient(at: x, y) { x, y in
    (x .* y).sum()
}

## その他の例

In [ ]:
let matrix = Tensor<Float>([[1, 2], [3, 4]])

print(_Raw.matMul(matrix, matrix, transposeA: true, transposeB: true))
print(_Raw.matMul(matrix, matrix, transposeA: true, transposeB: false))
print(_Raw.matMul(matrix, matrix, transposeA: false, transposeB: true))
print(_Raw.matMul(matrix, matrix, transposeA: false, transposeB: false))